# Argentina declared mandatory isolation with just ~100 confirmed cases
> Wasn't it too early? NO! with an exponential epidemic growth, it is never too early

- comments: false
- author: Pablo Zivic
- categories: [growth, compare, interactive]
- image: images/covid-growth-rate-confirmed-cases.png
- permalink: /covid-growth-rate-confirmed-cases/

In [ ]:
#hide

from io import StringIO
import csv
import pandas as pd
from datetime import datetime
import requests

BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'

def parse_data(fname):
    csv_content = requests.get(BASE_URL + fname).content
    docs = list(csv.DictReader(StringIO(csv_content.decode('utf8'))))
        
    new_docs = []
    for doc in docs:
        meta = {k: doc[k] for k in ['Province/State', 'Country/Region', 'Lat', 'Long']}
        for k, v in doc.items():
            if k in meta: continue
            new_doc = meta.copy()
            new_doc['date'] = datetime.strptime(k, '%m/%d/%y')
            new_doc['cnt'] = int(v)
            new_docs.append(new_doc)


    return (
        pd.DataFrame(new_docs)
          .rename(
              columns={
                  'Province/State': 'province',
                  'Country/Region': 'country',
                  'Lat': 'lat', 'Long': 'long'
              }
          ).groupby(['date', 'country'])
          .cnt.sum()
          .reset_index()
    )

In [4]:
#hide
confirmed_df = parse_data('csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
confirmed_df = confirmed_df.rename(columns=dict(cnt='confirmed'))

In [11]:
#hide
import numpy as np

dfs = []
for c in confirmed_df.country.unique():
    cdf = confirmed_df[confirmed_df.country == c].set_index('date').sort_index()
    cdf['rate'] = np.minimum(1.6, cdf.confirmed / cdf.confirmed.shift(1)).rolling(5).mean().fillna(1.6)
    cdf['cum'] = cdf.confirmed.cumsum()
    dfs.append(cdf)
    
confirmed_df = pd.concat(dfs).reset_index()

In [7]:
#hide
cnt_by_country = confirmed_df.groupby('country').confirmed.max().sort_values(ascending=False)

In [75]:
#hide
import altair as alt

def plot_rate_cnt_trajectories(countries, selected):
    selection = alt.selection_multi(
        fields=['country'], bind='legend', init=[{'country': c} for c in selected]
    )
    
    source = confirmed_df[(confirmed_df.confirmed > 30) & (confirmed_df.country.isin(countries))]
    
    return (
        alt.Chart(source)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X('confirmed', scale=alt.Scale(type='log'), axis=alt.Axis(title='Confirmed Cases')),
               y=alt.Y('rate', axis=alt.Axis(title='Rate of growth'), scale=alt.Scale(domain=[0.95, 1.65])),
               color='country:N',
               tooltip=list(confirmed_df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200).properties(width=650, height=400)
    )

On Thursday, March 19th the president of Argentina declared mandatory isolation for almost every Argentinian. 

Some might think it might have been unnecesary, given the fact that there were just only about 100 confirmed cases at that moment. 

I'm not going to tell you about the non syntomatic window where it is already contagious, instead I'm going to present a way of visualizing the dynamics of the evolution of Confirmed COVID-19 cases.

To read the figure:

- Each line corresponds to a country
- X Axis is the **total number of confirmed cases** at a given day
- Y Axis is the **rate at which the number of confirmed cases is growing**:  confirmed[t+1] / confirmed[t]

For example, Italy os now at 41k at 1.14 of growth rate. That means 40% more confirmed cases each day, meaning almost doubling it every 5 days.

In [73]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 
]

countries = list(cnt_by_country.index[:7]) + latam

plot_rate_cnt_trajectories(
    countries + ['Japan', 'Korea, South'], 
    selected=['Argentina', 'China', 'Italy', 'Japan', 'US', 'Korea, South']
)

alt.Chart(...)

In this plot you can compare the countries with the most affected, as well a come countries that took action and controled the spread.

- **China** and **South Korea** controlled the spread, both at 1 of growth rate
- **Japan** has a 3% daily growth rate, seems to have controlled the desease very effectively
- **Italy** and **China** curves are crossing, however Italy might not be deaccelerating as fast as China did

For now, it seems that Argentina's curve might be similar to that of Japan.

# What about the rest of Latin America?

In [76]:
#hide_input
latam = [
    'Brazil', 'Chile', 'Argentina', 'Panama', 'Colombia','Mexico', 
    'Ecuador', 'Costa Rica', 'Venezuela', 'Dominican Republic', 'Bolivia',
    'Paraguay', 'Uruguay','Honduras', 'Cuba', 'Puerto Rico','Guatemala', 'Japan'
]

plot_rate_cnt_trajectories(latam, ['Argentina', 'Chile', 'Brazil', 'Ecuador', 'Japan'])

alt.Chart(...)

All of Latam countries have a small amount of cases, so the Y axis corresponding to growth rate has a higher variance. However, given the current policies and growth rate we can have an idea.

- **Brazil** is growing fast, needs to take action quickly

- **Chile** seems to be starting a deacceleration, however it is too soon to tell

- **Argentina** is growing constantly at a 20% rate, but it is just about time to catch up to be like Japan


I truly hope these kind of charts help all citizens of the world adopt the required measures to fight the desease